# Retirement Planning Tool

This tool simulates a series of cashflows against historical market returns. 

This approach is a generalization of the approach behind the 4% rule. The 4% rule does not account for changes in your cashflow needs such as the start of Social Security, and assumes a 30 year timeframe. This tool allows you to choose your timeframe and models uneven cashflows. 

## Disclaimer

The information in this repo is provided "as is" for general informational purposes only. We make no warranties regarding its accuracy or completeness. We are not liable for any losses or damages resulting from your use of or reliance on this information. Use at your own risk.

In [1]:
import numpy as np
import xlrd

def load_data():
    book = book = xlrd.open_workbook("ie_data.xls")
    sheet = book.sheet_by_name("Data")
    dates = sheet.col_values(0) # Date
    prices = sheet.col_values(9) # Real Total Return Price
    assert len(dates) == 1853, len(dates)
    assert len(prices) == 1853, len(dates)
    return dates, prices

dates, prices = load_data()


In [2]:
def etl(dates, prices):
    """Cleanse the data: Trim the headers and excess white space at the end."""

    print(dates[:10])
    print(dates[-10:])

    print(prices[:10])
    print(prices[-10:])

    dates = dates[8:-1]
    prices = prices[8:-1]

    print(dates[0], dates[-10:])
    print(prices[0], prices[-10:])


    N = len(dates)

    print("Data consists of", N, "entries from", dates[0], "to", dates[-1])

    for idx, p in enumerate(prices):
        try: 
            float(p)
        except ValueError:
            print("Error: ", idx, p)


    dates = np.array(dates, dtype=np.float32)
    prices = np.array(prices, dtype=np.float32)

    return dates, prices

dates, prices = etl(dates, prices)

['', 'Stock Market Data Used in "Irrational Exuberance" Princeton University Press, 2000, 2005, 2015, updated', 'Robert J. Shiller ', '', '', '', '', 'Date', 1871.01, 1871.02]
[2023.12, 2024.01, 2024.02, 2024.03, 2024.04, 2024.05, 2024.06, 2024.07, 2024.08, '']
['', '', '', '', 'Real', 'Total', 'Return', 'Price', 111.94505242623511, 110.6262185217641]
[3172216.258905917, 3246908.2759480295, 3362452.0564336954, 3450516.754294489, 3402471.1361407707, 3482336.4999193936, 3604784.4188651205, 3685951.498636391, 3624862.5632474283, '']
1871.01 [2023.11, 2023.12, 2024.01, 2024.02, 2024.03, 2024.04, 2024.05, 2024.06, 2024.07, 2024.08]
111.94505242623511 [3013110.7220989885, 3172216.258905917, 3246908.2759480295, 3362452.0564336954, 3450516.754294489, 3402471.1361407707, 3482336.4999193936, 3604784.4188651205, 3685951.498636391, 3624862.5632474283]
Data consists of 1844 entries from 1871.01 to 2024.08


In [4]:
def calculate_equity_returns(prices):
    returns = [None] * (len(prices) - 1)
    for i in range (1, len(prices)):
        returns[i-1] = (prices[i] - prices[i-1]) / prices[i-1]
    return returns

equity_returns = calculate_equity_returns(prices)
print(np.mean(equity_returns))
print(np.std(equity_returns))

0.0064776065
0.040820498


In [19]:
def calculcate_portfolio_returns(equity_returns, annual_tips_returns=0.02, equity_ratio=0.5, tips_ratio=0.5):
    """Blend equities and bonds returns for a portfolio. Assumes regular rebalancing."""
    portfolio_returns = [None] * len(equity_returns)
    for i in range(len(equity_returns)):
        portfolio_returns[i] = equity_ratio * equity_returns[i] + tips_ratio * annual_tips_returns / 12.0
    return portfolio_returns

portfolio_returns = calculcate_portfolio_returns(equity_returns)

In [20]:
def simulate(cashflows, dates, returns):
    """Simulate the value of a portfolio given historical prices and cashflows.

    Args:
        cashflows: list of cashflows. The zero index should be positive to represent your retirement funds.
        historical_prices: list of historical equity prices from Schiller. 
    """
        
    C = len(cashflows)
    R = len(returns)

    assert C > 1, "Need at least two cashflows"

    num_sims = R - C + 1

    assert num_sims > 0, f"Cashflows must be less than returns. C={C}, R={R}"

    wins = 0
    losses = 0
    total = 0
    bad_retirement_years = []
    months_before_bankrupt = []
    for i in range(num_sims):
        portfolio = cashflows[0]
        assert portfolio > 0, "Initial portfolio must be positive"

        for c in range(1, C):
            portfolio += cashflows[c] # Spend that money.
            portfolio *= (1 + returns[i+c-1]) # Get a return on that money.
            if portfolio < 0:
                losses += 1
                total +=1
                bad_retirement_years.append(dates[i])
                months_before_bankrupt.append(c)
                break
        else: # no break
            wins += 1
            total += 1

    return wins, losses, total, bad_retirement_years, months_before_bankrupt

def test_simulate():
    # global: dates
    # global: portfolio_returns

    # Arrange
    dates_slice = dates[660:1465] # 1926 through 1992, like the original study
    returns_slice = portfolio_returns[660:1465] 
    savings = 100.0
    spend = -savings * 0.04 / 12 # Spend 4% annual or 0.33% monthly in real terms. 
    num_periods = 50 * 12 # The original paper modeld 50 years. 
    cashflows = [savings] + [spend] * num_periods

    # Act
    wins, losses, total, bad_retirement_years, months_before_bankrupt = simulate(cashflows, dates_slice, returns_slice)

    # Assert
    assert wins + losses == total
    assert 0.7 < wins / total <= 1.0, f"{wins} / {total} = {wins / total}"

    print("Test passed: success rate for 30 years of 4% rule is:", wins / total)
    print("The bad retirement years are:", bad_retirement_years)
    print("The months of retirement are:", months_before_bankrupt)

    # note that we do not find that a retiree would run out of money if retiring in 1969
    # despite a terrible sequence of returns in the 1970s becuase the US now offers
    # TIPS, which would not lose money. A regulatory / governance risk for your
    # retirement planning is if the US Govt eliminiates TIPS, or more likely, fudges
    # the calculation of inflation that TIPS are based on.

test_simulate()

Test passed: success rate for 30 years of 4% rule is: 0.9560975609756097
The bad retirement years are: [1929.03, 1929.04, 1929.05, 1929.06, 1929.07, 1929.08, 1929.09, 1929.1, 1930.04]
The months of retirement are: [582, 582, 575, 564, 498, 452, 422, 513, 561]
